In [11]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [12]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [13]:
soup_corona = bs(html_corona, 'html.parser')

In [14]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [15]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

In [16]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Date
0,World,2483181,2678.0,170501.0,104.0,652761.0,1659919,57338.0,319.00,21.9,0.0,0.0,2020-04-21 03:23:53.900228
1,USA,792938,179.0,42518.0,4.0,72389.0,678031,13951.0,2396.00,128.0,4027367.0,12167.0,2020-04-21 03:23:53.900228
2,Spain,200210,0.0,20852.0,0.0,80587.0,98771,7371.0,4282.00,446.0,930230.0,19896.0,2020-04-21 03:23:53.900228
3,Italy,181228,0.0,24114.0,0.0,48877.0,108237,2573.0,2997.00,399.0,1398024.0,23122.0,2020-04-21 03:23:53.900228
4,France,155383,0.0,20265.0,0.0,37409.0,97709,5683.0,2380.00,310.0,463662.0,7103.0,2020-04-21 03:23:53.900228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,South Sudan,4,0.0,0.0,0.0,0.0,4,0.0,0.40,0.0,0.0,0.0,2020-04-21 03:23:53.900228
209,Anguilla,3,0.0,0.0,0.0,1.0,2,0.0,200.00,0.0,0.0,0.0,2020-04-21 03:23:53.900228
210,Saint Pierre Miquelon,1,0.0,0.0,0.0,0.0,1,0.0,173.00,0.0,0.0,0.0,2020-04-21 03:23:53.900228
211,Yemen,1,0.0,0.0,0.0,0.0,1,0.0,0.03,0.0,0.0,0.0,2020-04-21 03:23:53.900228


In [17]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,...,Tests/ 1M pop,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Spain,200210,0.0,20852.0,0.0,80587.0,98771,7371.0,4282.0,446.0,...,19896.0,2020-04-21 03:23:53.900228,-3.647550,40.244487,101439.0,0.205562,0.104151,0.123624,0.012875,Last 8 columns are added by Roaya
2,Italy,181228,0.0,24114.0,0.0,48877.0,108237,2573.0,2997.0,399.0,...,23122.0,2020-04-21 03:23:53.900228,12.070013,42.796626,72991.0,0.330369,0.133059,0.088954,0.011836,Last 8 columns are added by Roaya
3,France,155383,0.0,20265.0,0.0,37409.0,97709,5683.0,2380.0,310.0,...,7103.0,2020-04-21 03:23:53.900228,2.618787,47.824905,57674.0,0.351372,0.130420,0.070287,0.009167,Last 8 columns are added by Roaya
0,USA,792938,179.0,42518.0,4.0,72389.0,678031,13951.0,2396.0,128.0,...,12167.0,2020-04-21 03:23:53.900228,-97.922211,39.381266,114907.0,0.370021,0.053621,0.140037,0.007509,Last 8 columns are added by Roaya
177,China,82758,11.0,4632.0,0.0,77123.0,1003,82.0,57.0,3.0,...,0.0,2020-04-21 03:23:53.900228,103.819074,36.561765,81755.0,0.056657,0.055970,0.099635,0.005577,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Guinea-Bissau,50,0.0,0.0,0.0,3.0,47,0.0,25.0,0.0,...,762.0,2020-04-21 03:23:53.900228,-14.949724,12.047449,3.0,0.000000,0.000000,0.000004,0.000000,Last 8 columns are added by Roaya
141,Uganda,56,0.0,0.0,0.0,38.0,18,0.0,1.0,0.0,...,300.0,2020-04-21 03:23:53.900228,32.369080,1.274693,38.0,0.000000,0.000000,0.000046,0.000000,Last 8 columns are added by Roaya
122,Madagascar,121,0.0,0.0,0.0,41.0,80,1.0,4.0,0.0,...,85.0,2020-04-21 03:23:53.900228,46.704737,-19.371896,41.0,0.000000,0.000000,0.000050,0.000000,Last 8 columns are added by Roaya
135,Maldives,69,0.0,0.0,0.0,16.0,53,1.0,128.0,0.0,...,7657.0,2020-04-21 03:23:53.900228,73.457130,3.728709,16.0,0.000000,0.000000,0.000019,0.000000,Last 8 columns are added by Roaya


In [18]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [19]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [20]:
corona_json

'[{"Country":"USA","TotalCases":792938,"NewCases":179.0,"TotalDeaths":42518.0,"NewDeaths":4.0,"TotalRecovered":72389.0,"ActiveCases":678031,"CriticalCases":13951.0,"Tot\\u00a0Cases\\/1M pop":2396.0,"Deaths\\/1M pop":128.0,"TotalTests":4027367.0,"Tests\\/ 1M pop":12167.0,"Date":1587439433900,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":114907.0,"ClosedDeathRate":0.3700209735,"CurrentDeathRate":0.0536208379,"Weight":0.1400370728,"DeathRateXWeight":0.0075089052,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spain","TotalCases":200210,"NewCases":0.0,"TotalDeaths":20852.0,"NewDeaths":0.0,"TotalRecovered":80587.0,"ActiveCases":98771,"CriticalCases":7371.0,"Tot\\u00a0Cases\\/1M pop":4282.0,"Deaths\\/1M pop":446.0,"TotalTests":930230.0,"Tests\\/ 1M pop":19896.0,"Date":1587439433900,"Longitude":-3.64755047,"Latitude":40.24448698,"ClosedCases":101439.0,"ClosedDeathRate":0.2055619633,"CurrentDeathRate":0.1041506418,"Weight":0.1236236316,"DeathRateXWeight":0.0128754806,"note